#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter CM Report Emailed To BigQuery Parameters
Pulls a CM Report from a gMail powered email account into BigQuery.
 1. The person executing this recipe must be the recipient of the email.
 1. Schedule a CM report to be sent to <b>UNDEFINED</b>.
 1. Or set up a redirect rule to forward a report you already receive.
 1. The report must be sent as an attachment.
 1. Ensure this recipe runs after the report is email daily.
 1. Give a regular expression to match the email subject.
 1. Configure the destination in BigQuery to write the data.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for reading data.
  'email': '',  # Email address report was sent to.
  'subject': '.*',  # Regular expression to match subject. Double escape backslashes.
  'dataset': '',  # Existing dataset in BigQuery.
  'table': '',  # Name of table to be written to.
  'is_incremental_load': False,  # Append report data to table based on date column, de-duplicates.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute CM Report Emailed To BigQuery
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'email': {
      'auth': 'user',
      'read': {
        'from': 'noreply-cm@google.com',
        'subject': {'field': {'description': 'Regular expression to match subject. Double escape backslashes.','name': 'subject','default': '.*','order': 2,'kind': 'string'}},
        'to': {'field': {'description': 'Email address report was sent to.','name': 'email','default': '','order': 1,'kind': 'string'}},
        'attachment': '.*',
        'out': {
          'bigquery': {
            'table': {'field': {'description': 'Name of table to be written to.','name': 'table','default': '','order': 4,'kind': 'string'}},
            'is_incremental_load': {'field': {'description': 'Append report data to table based on date column, de-duplicates.','name': 'is_incremental_load','default': False,'order': 6,'kind': 'boolean'}},
            'dataset': {'field': {'description': 'Existing dataset in BigQuery.','name': 'dataset','default': '','order': 3,'kind': 'string'}}
          }
        }
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
